# LOG6308 : Tp2 - Approche contenu

- Clément Bernard (2096223)
- Ghaith Dekhili (1858454)

## Importations 

In [24]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import os
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
DATA = os.path.join(os.getcwd(), 'data')
TABLE_PATH = os.path.join(DATA, 'citeseer.rtable')

## Data 


In [3]:
# Matrice d'adjacence
m = pd.read_table(TABLE_PATH, sep=' ')
# Number of articles
N = m.shape[0]

# Question 1

In [4]:
# TO DO

# Question 2

In [5]:
m

,100299,100967,10151,101705,101863,102458,102886,102966,10302,103700,...,96767,97060,97150,9721,97410,97863,98185,99113,9947,9993
100299,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10151,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
101705,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
101863,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97863,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98185,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99113,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9947,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


- Tout d'abord, pour éviter les divisions par 0, on met la diagonale de la matrice à 1

In [6]:
np.fill_diagonal(m.values, 1)

- Ensuite, nous appliquons l'algorithne PageRank

In [7]:
def page_rank(m , d = 0.85, epsilon = 1e-5, max_iter = 10000) : 
    ''' Compute the PageRank score '''
    # Number of articles 
    N = m.shape[0]
    # PageRank values
    r = np.ones((1,N))
    # Initialise the condition 
    new_r = r + 10
    # Iteration 
    iteration = 0
    # Condition : if the mean of difference is greater than epsilon
    condition = abs(r - new_r).mean() > epsilon 
    
    while condition and iteration < max_iter     : 
        # Get the number of output links 
        s = m.sum(axis=0).values.reshape(-1,1)
        # Get the new PageRank vector
        new_r = (1-d)/N +  d * m @ (r.T / s)                
        # Convert into numpy 
        new_r = new_r.values.T
        # Update the condition 
        condition = abs(r - new_r).mean() > epsilon

        r = new_r
        
        iteration+=1
        
    return pd.DataFrame(r.reshape(-1,), index = m.index, columns=['PageRank'])

In [8]:
r = page_rank(m, d= 0.85, epsilon = 1e-5, max_iter = 1000)

In [9]:
r

,PageRank
100299,0.001470
100967,0.000447
10151,0.000674
101705,0.001167
101863,0.000289
...,...
97863,0.006034
98185,0.000279
99113,0.002592
9947,0.000675


- Recommendation : prendre les articles maximum de PageRank

In [10]:
def get_recommendation(index, r = r, m=m,N= 10 ): 
    ''' Return recommendations for the index article from PageRank score '''
    # Get the element wise product between the PageRank and adjacent matrix
    idx = m[m.index == index].values * r.values.reshape(1,-1)
    idx = (-idx[0]).argsort()[:N]
    recommend = m.index[idx]
    return list(recommend)

In [11]:
get_recommendation(422908)

[422908, 155792, 3170, 131548, 241538, 17094, 124, 466838, 147460, 149673]

- Variante : etendre le sous-ensemble S aux références de références

In [12]:
m_ = m.values @ m.values + m.values.T

In [13]:
m_[m_ >1] = 1

In [14]:
new_m = m.copy()

In [15]:
m_

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [16]:
new_m[:] = m_

In [17]:
new_m

,100299,100967,10151,101705,101863,102458,102886,102966,10302,103700,...,96767,97060,97150,9721,97410,97863,98185,99113,9947,9993
100299,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100967,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10151,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
101705,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
101863,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97863,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
98185,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
99113,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9947,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [18]:
new_r = page_rank(new_m, d= 0.85, epsilon = 1e-5, max_iter = 1000)

In [19]:
def get_recommendation_new(index, r = new_r, m=new_m,N= 10 ): 
    ''' Return recommendations for the index article from PageRank score '''
    # Get the element wise product between the PageRank and adjacent matrix
    idx = m[m.index == index].values * r.values.reshape(1,-1)
    idx = (-idx[0]).argsort()[:N]
    recommend = m.index[idx]
    return list(recommend)

In [20]:
get_recommendation_new(422908)

[311874, 19422, 422908, 83263, 464834, 297641, 366858, 177173, 17507, 226359]

# Question 3

- On considère pour cette question qu'un utilisateur est représenté par son vecteur dans la matrice d'adjacence
- On calcule ensuite la distance cosinus pour chaque utilisateur 
- La première recommendation sera forcément l'utilisateur lui-même 

In [70]:
def get_recommendation_sim(m = m , N=10) :
    ''' Return the top N recommendation using the cosine similarity '''
    # Get the similarity for each user 
    df_sim = pd.DataFrame(cosine_similarity(m,m), index = m.index, columns = m.columns)
    # Get the indexes that maximises the cosine similarity
    idx = (-df_sim.values).argsort()
    # Keep the N max
    idx = idx[:,:N]
    
    return pd.DataFrame(idx)

In [72]:
get_recommendation_sim(N=10)

,0,1,2,3,4,5,6,7,8,9
0,0,1070,440,971,202,477,73,875,720,721
1,1,315,482,519,461,951,345,426,484,772
2,2,784,1068,109,282,718,720,721,728,722
3,3,759,53,1054,730,729,728,727,726,724
4,4,1005,116,355,358,970,149,439,121,96
...,...,...,...,...,...,...,...,...,...,...
1085,1085,233,496,227,1018,509,208,489,867,483
1086,1086,125,91,1031,721,237,117,810,677,967
1087,1087,1004,417,718,719,720,721,722,729,723
1088,1088,544,1082,135,1008,871,192,210,290,414


# Question 4

In [28]:
# TO DO

# Question 5

In [28]:
# TO DO